In [ ]:
import sys
import os
sys.path.append(os.path.abspath("../"))
sys.path.append(os.path.abspath("../.."))
from fashion_class.Coordinate import Coordinate
from fashion_class.ImageStruct import ImageStruct

In [ ]:
# データセットを初期化
annotation_file, tensor_file = \
      '../data/anotation_new.csv', '../data/resnet50_tensor.pt'
dataset = ImageStruct(annotation_file, tensor_file, init_item_length = 300000)

In [ ]:
tops = dataset.tops
bottoms = dataset.bottoms
shoes = dataset.shoes

In [ ]:
from fashion_class.FashionItem import FashionItem
import numpy as np
import tomotopy as tp
from lda_model.util.ignore_items import is_ignore_items
from lda_model.util.is_stopword import is_stopword
from preProcessing.preprocessing import preprocessing

from lda_model.util.parse_sentence import parse_sentence

# モデルを読み込む
mdl = tp.CTModel.load('ctm-T-10-M-10000-R-10-B-3000.bin')
def get_ctm_base_compatibility(t: FashionItem, b: FashionItem, s: FashionItem, mdl):
    doc = []
    for item in [t, b, s]:
        category = item.category.split(' × ')[0]
        if ("expressions" not in item or
                len(item["expressions"]) == 0 or
                category == "" or
                is_ignore_items(category)):
            continue
        rm_br_str = preprocessing(item.item_info["expressions"][0], debug=False)
        res = parse_sentence(rm_br_str)
        for word, hinshi in res:
            if is_stopword(hinshi) or len(word) <= 1:
                continue
            word = category + "_" + word
            doc += [word]
    if len(doc) == 0:
        return 0
    inf_doc = mdl.make_doc(doc)
    log_prob = mdl.infer(inf_doc, iter=500)[1]
    return log_prob


In [ ]:
c = 0
KEY = "category x color"
length = len(tops) * len(bottoms) * len(shoes)
print(length)
ctm_base = []
com = []
for t in tops:
    for b in bottoms:
        for s in shoes:
            c += 1
            coordinate = Coordinate(t, b, s)
            compatibility = coordinate.get_compatibility()
            com.append(compatibility)
            ctm_base.append(get_ctm_base_compatibility(t, b, s, mdl))
            if (c % 1000 == 0):
                print(f"\r {c * 100 // length}%終了", end='')
